In [2]:
from api.riot_api_routes import *
from api.variables import SERVER_ROUTES, SERVER_REAL_NAME_TO_ROUTE, DIVISIONS, TIERS, GAME_TYPE,MAIN_GAMEMODE,RANKED_QUEUES, MATCHES_STATS_KEYS

import json
from django.conf import settings
from os import path
import os
import random

import pandas as pd
from time import sleep

league_data_route=path.join(settings.PREPROCESS_PATH, "league_data.json")
player_sample_route=path.join(settings.PREPROCESS_PATH, "player_sample.json")
matches_sample_route=path.join(settings.PREPROCESS_PATH, "matches_sample.csv")
champ_ban_file=path.join(settings.PREPROCESS_PATH, "champ_ban.csv")
champ_data_file=path.join(settings.PREPROCESS_PATH, "champ_data.csv")
playstyle_data_file=path.join(settings.PREPROCESS_PATH, "playstyle_data.csv")


In [3]:
champ_data = pd.read_csv(champ_data_file)
champ_data

,championId,teamId,role,lane,spell1Id,spell2Id,gameDuration,win,item0,item1,...,perk1,perk2,perk3,perk4,perk5,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,2,100,DUO_SUPPORT,NONE,11,4,1159,False,3071,2031,...,9111,9104,8299,8304,8410,8000,8300,5008,5008,5001
1,119,100,DUO_SUPPORT,NONE,4,7,1159,False,1055,3072,...,9111,9103,8014,8233,8236,8000,8200,5005,5008,5002
2,68,100,DUO_SUPPORT,NONE,14,4,1159,False,3020,1052,...,8275,8234,8237,8139,8135,8200,8100,5008,5008,5003
3,103,100,DUO_SUPPORT,NONE,4,14,1159,False,3859,3030,...,8304,8345,8347,8139,8134,8300,8100,5007,5003,5002
4,6,100,DUO_SUPPORT,NONE,4,12,1159,False,1055,3047,...,9111,9105,8299,8444,8451,8000,8400,5008,5008,5002
5,107,200,DUO_SUPPORT,NONE,4,11,1159,True,3077,3147,...,9111,9104,8014,8143,8105,8000,8100,5005,5008,5002
6,143,200,DUO_SUPPORT,NONE,4,14,1159,True,3853,2055,...,8226,8210,8237,8139,8105,8200,8100,5008,5008,5002
7,81,200,DUO_SUPPORT,NONE,7,4,1159,True,0,3004,...,8009,9103,8014,8226,8210,8000,8200,5005,5008,5002
8,112,200,DUO_SUPPORT,NONE,4,12,1159,True,1052,3802,...,8226,8210,8236,8304,8321,8200,8300,5008,5008,5003
9,114,200,DUO_SUPPORT,NONE,4,12,1159,True,3047,3074,...,9111,9104,8299,8345,8352,8000,8300,5005,5008,5002


In [7]:
len(champ_data)

50

In [8]:
champ_data['championId'].value_counts()

267    3
64     2
51     2
875    2
40     2
21     2
34     2
81     2
59     2
16     1
240    1
555    1
86     1
25     1
26     1
27     1
75     1
6      1
157    1
4      1
39     1
63     1
154    1
126    1
61     1
60     1
57     1
56     1
119    1
54     1
498    1
79     1
48     1
412    1
145    1
107    1
105    1
103    1
38     1
36     1
Name: championId, dtype: int64

In [7]:
champ_ban = pd.read_csv(champ_ban_file)
champ_ban.head()

,championId,pickTurn,division,tier,win
0,53,1,SILVER,IV,True
1,35,2,SILVER,IV,True
2,122,3,SILVER,IV,True
3,150,4,SILVER,IV,True
4,157,5,SILVER,IV,True


In [6]:
champ_ban['championId'].value_counts()

 350    3
 131    3
 516    3
 238    3
 245    3
-1      2
 60     2
 89     2
 111    2
 142    2
 55     2
 8      2
 266    1
 17     1
 67     1
 23     1
 24     1
 154    1
 157    1
 62     1
 31     1
 875    1
 53     1
 38     1
 105    1
 30     1
 235    1
 51     1
 54     1
 56     1
 420    1
 122    1
 35     1
Name: championId, dtype: int64

In [24]:
winrate_on_ban=[]
for champ in champ_ban['championId'].unique():
    total = len(champ_ban.loc[(champ_ban['championId'] == champ)])
    win = len(champ_ban.loc[(champ_ban['championId'] == champ) & (champ_ban['win']==True)])
    winrate = win * 100 / total
    winrate_on_ban.append({
        "championId":champ,
        "winrate":winrate,
        "total":total
    })
pd.DataFrame(winrate_on_ban)

,championId,winrate,total
0,105,100.000000,1
1,122,100.000000,1
2,8,100.000000,2
3,350,66.666667,3
4,55,50.000000,2
5,238,66.666667,3
6,154,0.000000,1
7,131,33.333333,3
8,516,33.333333,3
9,24,100.000000,1


1

In [ ]:
winrate_on_ban['winrate']= * 100 / len(winrate_on_ban[])

In [19]:
len(winrate_on_ban['championId'])

50

In [6]:
psd = pd.read_csv(playstyle_data_file)
psd

,championId,teamId,role,lane,gameDuration,tier,division,win,kills,assists,...,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,goldEarned,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,wardsPlaced,wardsKilled
0,201,100,DUO_SUPPORT,BOTTOM,1258,IV,SILVER,True,2,20,...,6712,5030,1294,6248,23,0,0,0,8,2
1,161,100,SOLO,MIDDLE,1258,IV,SILVER,True,6,12,...,23061,15046,1467,9051,129,4,0,0,7,2
2,22,100,DUO_CARRY,BOTTOM,1258,IV,SILVER,True,12,13,...,14988,1650,13338,11219,123,4,0,4,5,0
3,83,100,SOLO,TOP,1258,IV,SILVER,True,3,5,...,9436,2808,6628,7080,100,0,0,0,6,4
4,72,100,NONE,JUNGLE,1258,IV,SILVER,True,10,3,...,6839,1500,4549,8895,8,84,37,23,4,1
5,39,200,SOLO,MIDDLE,1258,IV,SILVER,False,3,2,...,8423,2773,4960,7046,127,0,0,0,5,0
6,114,200,SOLO,TOP,1258,IV,SILVER,False,3,2,...,10014,640,7289,7927,124,0,0,0,8,1
7,121,200,NONE,JUNGLE,1258,IV,SILVER,False,2,1,...,3502,273,2816,5759,7,71,44,8,9,3
8,412,200,DUO_SUPPORT,BOTTOM,1258,IV,SILVER,False,3,4,...,6205,4617,852,5497,23,0,0,0,12,1
9,21,200,DUO_CARRY,BOTTOM,1258,IV,SILVER,False,2,3,...,9351,696,8548,7100,117,0,0,0,6,0
